In [1]:
import utils.constants as constants
import utils.text_processing as text_processing
import utils.ranking as ranking
import utils.multi_ranking as multi_ranking
import utils.text_processing as text_processing

import umap
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler


In [2]:
embeddings, dim = text_processing.load_embeddings(constants.STARSPACE_EMBEDDINGS)
paper = {"abstract": "We propose to use Rademacher complexity, originally developed in computational learning theory, as a measure of human learning capacity. Rademacher complex- ity measures a learner’s ability to fit random labels, and can be used to bound the learner’s true error based on the observed training sample error. We first re- view the definition of Rademacher complexity and its generalization bound. We then describe a “learning the noise” procedure to experimentally measure human Rademacher complexities. The results from empirical studies showed that: (i) human Rademacher complexity can be successfully measured, (ii) the complex- ity depends on the domain and training sample size in intuitive ways, (iii) hu- man learning respects the generalization bounds, (iv) the bounds can be useful in predicting the danger of overfitting in human learning. Finally, we discuss the potential applications of human Rademacher complexity in cognitive science."}

In [ ]:
%%time
ranked = multi_ranking.rank_candidates_per_subject(paper, 10, embeddings, dim, constants.SUBJECTS, False, True)

In [6]:
%%time
ranked = ranking.rank_candidates_per_subject(paper, 10, embeddings, dim, constants.SUBJECTS, False, True)

In [6]:
citations = ranking.build_citation_neighborhood("395de0bd3837fdf4b4b5e5f04835bcc69c279481")
data = pd.DataFrame.from_dict(citations, orient="index")

In [45]:
embeddings, dim = text_processing.load_embeddings(constants.STARSPACE_EMBEDDINGS)

In [101]:
ranked = ranking.rank_citation_neighborhood("395de0bd3837fdf4b4b5e5f04835bcc69c279481", {"abstract": "We present BART, a denoising autoencoder for pretraining sequence-to-sequence models. BART is trained by (1) corrupting text with an arbitrary noising function, and (2) learning a model to reconstruct the original text. It uses a standard Tranformer-based neural machine translation architecture which, despite its simplicity, can be seen as generalizing BERT (due to the bidirectional encoder), GPT (with the left-to-right decoder), and many other more recent pretraining schemes. We evaluate a number of noising approaches, finding the best performance by both randomly shuffling the order of the original sentences and using a novel in-filling scheme, where spans of text are replaced with a single mask token. BART is particularly effective when fine tuned for text generation but also works well for comprehension tasks. It matches the performance of RoBERTa with comparable training resources on GLUE and SQuAD, achieves new state-of-the-art results on a range of abstractive dialogue, question answering, and summarization tasks, with gains of up to 6 ROUGE. BART also provides a 1.1 BLEU increase over a back-translation system for machine translation, with only target language pretraining. We also report ablation experiments that replicate other pretraining schemes within the BART framework, to better measure which factors most influence end-task performance."}, 100, embeddings, dim, True, True)

In [ ]:
ranked = ranking.rank_candidates_adjusted(paper, 1000, embeddings, dim, constants.SUBJECTS)
data_ranked = pd.DataFrame(ranked, columns=["ID", "FILE", "DISTANCE", "SUBJECT", "TITLE", "ABSTRACT", "EMBEDDING"])

In [ ]:
umap_reducer2D = umap.UMAP(n_components=2)

data_ranked["SIMILARITY"] = 1 - MinMaxScaler().fit_transform(data_ranked["DISTANCE"].values.reshape((-1, 1)))
data_ranked["SIMILARITY"] = data_ranked["SIMILARITY"].clip(lower=0)

initial_embeddings = np.stack(data_ranked["EMBEDDING"])
mapped_embeddings = umap_reducer2D.fit_transform(initial_embeddings)

data_ranked["X"] = mapped_embeddings[:, 0]
data_ranked["Y"] = mapped_embeddings[:, 1]

In [ ]:
fig = px.scatter(data_ranked, x="X", y="Y", color = "SUBJECT", height=800, hover_name="TITLE", size="SIMILARITY")
fig.show()